In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2


2026-02-05 15:41:03.128745: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv('coffee_beans.csv')
X = df["filepaths"]
y = df["class index"]


In [3]:
def load_images(x):
    images = []
    for path in x:
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224, 224))
        img = img / 255.0  # normalize
    
        images.append(img)
    
    images = np.array(images, dtype=np.float32)
    return images



Split dataset into training, validation and test set. This split will be used to choose the best neural network architecture for the final model.

In [4]:
from sklearn.model_selection import train_test_split

X_train, x_, y_train, y_ = train_test_split(X , y, test_size=0.40, random_state=1)
X_cv, X_test, y_cv, y_test = train_test_split(x_ , y_, test_size=0.20, random_state=1)

del x_, y_

Load the images defined in the $x$ sets.

In [5]:
X_train = load_images(X_train)
X_cv = load_images(X_cv)
X_test = load_images(X_test)

In [6]:
tf.keras.utils.set_random_seed(42) # seed for reproducibility

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten

model_1 = Sequential([
    Flatten(input_shape=(224, 224, 3)), # ??? have to research
    Dense(units=25, activation='relu'),
    Dense(units=15, activation='relu'),
    Dense(4, activation='linear'), # since the from_logits is used in the loss function
], name="CoffeeRoastAI_1")

model_2 = Sequential([
    Flatten(input_shape=(224, 224, 3)), # ??? have to research
    Dense(units=15, activation='relu'),
    Dense(units=15, activation='relu'),
    Dense(4, activation='linear'), # since the from_logits is used in the loss function
], name="CoffeeRoastAI_2")

model_3 = Sequential([
    Flatten(input_shape=(224, 224, 3)), # ??? have to research
    Dense(units=5, activation='relu'),
    Dense(units=15, activation='relu'),
    Dense(4, activation='linear'), # since the from_logits is used in the loss function
], name="CoffeeRoastAI_3")

nn_models = [model_1, model_2, model_3]

/home/dflorjan/Documents/coffee-roast-ai/venv/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
from sklearn.metrics import log_loss

nn_train_cross_entropy = []
nn_cv_cross_entropy = []

for model in nn_models:
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), # gradient descent optimatiation
        loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'], # ??? have to research
    )
    print(f"Training {model.name}...")
    
    model.fit(X_train, y_train, epochs=10)
    print("Done\n")

    # Instantiate loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

    # Record the training Log Losses
    yhat = model.predict(X_train)
    pred_class = np.argmax(yhat, axis=1)[0]
    train_cross_entropy = loss_fn(y_train, yhat)
    nn_train_cross_entropy.append(train_cross_entropy)

    # Record the cross validation Log Losses
    yhat = model.predict(X_cv)
    pred_class = np.argmax(yhat, axis=1)[0]
    cv_cross_entropy = loss_fn(y_cv, yhat)
    nn_cv_cross_entropy.append(cv_cross_entropy)

Training CoffeeRoastAI_1...
Epoch 1/10


2026-02-05 15:45:13.642560: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 578027520 exceeds 10% of free system memory.


30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 0.2844 - loss: 18.4683
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.4833 - loss: 2.0794
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.5833 - loss: 1.0138
Epoch 4/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.5906 - loss: 0.9689
Epoch 5/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.6031 - loss: 0.9349
Epoch 6/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.6073 - loss: 0.9217
Epoch 7/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.6115 - loss: 0.9020
Epoch 8/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.6229 - loss: 0.8947
Epoch 9/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.6240 - loss: 0.9408
Epoch 10/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.6271 - loss: 0.9326
Done

 6/30 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step

2026-02-05 15:45:30.338041: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 578027520 exceeds 10% of free system memory.


30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
10/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step

...

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Training CoffeeRoastAI_2...
Epoch 1/10


2026-02-05 15:45:32.064780: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 578027520 exceeds 10% of free system memory.


30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - accuracy: 0.2406 - loss: 2.0261
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.2427 - loss: 1.3864
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.2604 - loss: 1.3863
Epoch 4/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.2604 - loss: 1.3862
Epoch 5/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.2604 - loss: 1.3861
Epoch 6/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.2604 - loss: 1.3861
Epoch 7/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.2604 - loss: 1.3860
Epoch 8/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.2604 - loss: 1.3860
Epoch 9/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.2604 - loss: 1.3859
Epoch 10/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.2604 - loss: 1.3859
Done

14/30 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step

2026-02-05 15:45:43.381365: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 578027520 exceeds 10% of free system memory.


30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Training CoffeeRoastAI_3...
Epoch 1/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.2385 - loss: 3.9340
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.2479 - loss: 1.3864
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.2333 - loss: 1.3864
Epoch 4/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.2531 - loss: 1.3863
Epoch 5/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.2531 - loss: 1.3862
Epoch 6/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.2531 - loss: 1.3861
Epoch 7/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.2531 - loss: 1.3861
Epoch 8/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.2531 - loss: 1.3861
Epoch 9/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.2438 - loss: 1.3860
Epoch 10/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.2490 - loss: 1.3860
Done

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/st

In [10]:
print("RESULTS:")
for model_num in range(len(nn_train_cross_entropy)):
    print(
        f"Model {model_num+1}: Training MSE: {nn_train_cross_entropy[model_num]:.2f}, " +
        f"CV MSE: {nn_train_cross_entropy[model_num]:.2f}"
        ) 

RESULTS:
Model 1: Training MSE: 1.38, CV MSE: 1.38
Model 2: Training MSE: 4.37, CV MSE: 4.37
Model 3: Training MSE: 6.53, CV MSE: 6.53


Because softmax is integrated into the output layer the output s a vector of probabilities.

Make predictions

In [11]:
logits = model(X_train) # outpus z_1, .., z_n and not a_1, .., a_n
f_x = tf.nn.softmax(logits) # map the z result to softmax function
f_x

<tf.Tensor: shape=(960, 4), dtype=float32, numpy=
array([[0.24936813, 0.24330074, 0.2541439 , 0.2531872 ],
       [0.24936813, 0.24330074, 0.2541439 , 0.2531872 ],
       [0.24936813, 0.24330074, 0.2541439 , 0.2531872 ],
       ...,
       [0.24936813, 0.24330074, 0.2541439 , 0.2531872 ],
       [0.24936813, 0.24330074, 0.2541439 , 0.2531872 ],
       [0.24936813, 0.24330074, 0.2541439 , 0.2531872 ]],
      shape=(960, 4), dtype=float32)>

In [12]:
class_names = {
    0: "Dark",
    1: "Green",
    2: "Light",
    3: "Medium"
}


In [20]:
IMG_SIZE = 224

img = cv2.imread("test/Green/green (20).png")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
img = img / 255.0

img = np.expand_dims(img, axis=0)  # shape (1, 224, 224, 3)

pred = model.predict(img)
pred_class = np.argmax(pred, axis=1)[0]

print("Predicted:", class_names[pred_class])
print("Class probabilities (Dark, Green, Light, Medium):", pred[0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Predicted: Light
Class probabilities (Dark, Green, Light, Medium): [-0.00091913 -0.02555107  0.01805125  0.01427975]


Save the model

In [ ]:
model.save("coffee_roast_model.keras")